# libCEED for Python examples

This is a tutorial to illustrate the main feautures of the Python interface for [libCEED](https://github.com/CEED/libCEED/), the low-level API library for efficient high-order discretization methods developed by the co-design [Center for Efficient Exascale Discretizations](https://ceed.exascaleproject.org/) (CEED) of the [Exascale Computing Project](https://www.exascaleproject.org/) (ECP).

While libCEED's focus is on high-order finite/spectral element method implementations, the approach is mostly algebraic and thus applicable to other discretizations in factored form, as explained in the [user manual](https://libceed.org/).

## Setting up libCEED for Python

Install libCEED for Python by running

In [ ]:
! python -m pip install libceed

## CeedVector

Here we show some basic examples to illustrate the `libceed.Vector` class. In libCEED, CeedVectors constitute the main data structure and serve as input/output for `libceed.Operator`. 

We illustrate the simple creation of a `libceed.Vector`, how to specify its size, and how to read or manipulate its data.

In [ ]:
import libceed

ceed = libceed.Ceed()

n = 10
x = ceed.Vector(n)

The size of the `CeedVector` can also be specified as 

In [ ]:
x = ceed.Vector(size=10)

* In the following example, we associate the data stored in a `libceed.Vector` with a `numpy.array` and use it to set and read the `libceed.Vector`'s data

In [ ]:
import numpy as np

ceed = libceed.Ceed()
x = ceed.Vector(size=3)

a = np.arange(1, 4, dtype="float64")
x.set_array(a, cmode=libceed.USE_POINTER)

with x.array_read() as b:
  print(b)

* In the following example, we set all entries to the same value and then visualize the `libceed.Vector`

In [ ]:
ceed = libceed.Ceed()
x = ceed.Vector(size=5)

x.set_value(10)

print(x)

* In the following example, we set one vector from the array of another vector

In [ ]:
ceed = libceed.Ceed()
n = 10

x = ceed.Vector(n)
y = ceed.Vector(n)

a = np.arange(1, 1 + n, dtype="float64")
x.set_array(a, cmode=libceed.USE_POINTER)

with x.array() as x_array:
  y.set_array(x_array, cmode=libceed.USE_POINTER)

with y.array_read() as y_array:
  print(y_array)


* In the following example, we access and modify only one entry of the `libceed.Vector`

In [ ]:
n = 10

x = ceed.Vector(n)
a = np.zeros(n, dtype="float64")
x.set_array(a, cmode=libceed.USE_POINTER)

with x.array() as b:
  b[3] = -3.14;
a[3]

* In the following example, we compute the $L_1$, $L_2$ (default), and $L_{\infty}$ norms of a `libceed.Vector` (keeping in mind that these are local norm computations; not accurate for parallel executions with no reductions).

In [ ]:
n = 10
x = ceed.Vector(n)

a = np.arange(0, n, dtype="float64")
for i in range(n):
  if (i % 2 == 0):
    a[i] *= -1
x.set_array(a, cmode=libceed.USE_POINTER)

norm_1 = x.norm(normtype=libceed.NORM_1)
print(norm_1)

norm_2 = x.norm()
print(norm_2)

norm_max = x.norm(normtype=libceed.NORM_MAX)
print(norm_max)

* If you have installed libCEED with CUDA support and Numba, you can use device memory in your `libceed.Vector`s. In the following example, we create a `libceed.Vector` with a libCEED contex that supports CUDA, associate the data stored in a CeedVector with a `numpy.array`, and get a Numba `DeviceNDArray` containing the data on the device.

In [ ]:
ceed_gpu = libceed.Ceed('/gpu/cuda')

n = 10
x = ceed_gpu.Vector(n)

a = np.arange(1, 1 + n, dtype="float64")
x.set_array(a, cmode=libceed.USE_POINTER)

with x.array_read(memtype=libceed.MEM_DEVICE) as device_array:
    print(device_array)